In [1]:
import os
import glob
import pandas as pd
import matplotlib.pyplot as plt
import pyarrow as pa
import pyarrow.parquet as pq
import pandas as pd
import datetime
from pandas.tseries.holiday import USFederalHolidayCalendar as calendar

Below I have code to collect all the parquet files in a list and than concataning the files together and printing them in a wrong way. But because I didn't want to keep doing it. It is commented out. I than have the file being read into them.Than I wanted to check shape and columns.

In [2]:
cleaneddata = pd.read_parquet('cleaneddata.parquet')
cleaneddata.shape

(37564080, 23)

## Manhattan zones 

In [3]:
# Load the data for taxi zone
zones = pd.read_csv('taxi_zones.csv')
zones.shape

(263, 7)

In [4]:
# Filter for Manhattan
manhattan_zones = zones[zones['borough'] == 'Manhattan']
manhattan_zones[manhattan_zones["LocationID"] == 103]

,OBJECTID,Shape_Leng,the_geom,Shape_Area,zone,LocationID,borough
103,103,0.014306,MULTIPOLYGON (((-74.04388559600675 40.69018482...,0.000006,Governor's Island/Ellis Island/Liberty Island,103,Manhattan
104,104,0.021221,MULTIPOLYGON (((-74.03995040794244 40.70089063...,0.000012,Governor's Island/Ellis Island/Liberty Island,103,Manhattan
105,105,0.077425,MULTIPOLYGON (((-74.01674756096064 40.69334336...,0.000369,Governor's Island/Ellis Island/Liberty Island,103,Manhattan


In [5]:
# Extract location IDs
manhattan_ids = manhattan_zones['LocationID'].tolist()
manhattan_ids = [x for x in manhattan_ids if x != 103]
print(manhattan_ids)

[4, 24, 12, 13, 41, 45, 42, 43, 48, 50, 68, 79, 74, 75, 87, 88, 90, 125, 100, 107, 113, 114, 116, 120, 127, 128, 151, 140, 137, 141, 142, 152, 143, 144, 148, 153, 158, 161, 162, 163, 164, 170, 166, 186, 194, 202, 209, 211, 224, 229, 230, 231, 239, 232, 233, 234, 236, 237, 238, 263, 243, 244, 246, 249, 261, 262]


In [6]:
# Filter the dataframe to drop rows where neither PULocationID nor DOLocationID is in the Manhattan ID list
cleaneddata = cleaneddata[cleaneddata['PULocationID'].isin(manhattan_ids) | cleaneddata['DOLocationID'].isin(manhattan_ids)]

## All below are just for analysing the data. 
i broke them down into months, hours, and season, holidays. 

Ok firstly I want the month and I just want to order them because it wasnt ordered in mine. And change it to category this is just for analysis though.

In [7]:
# Extract the month from the datetime columns
cleaneddata['pickup_month'] = cleaneddata['tpep_pickup_datetime'].dt.month_name()
cleaneddata['dropoff_month'] = cleaneddata['tpep_dropoff_datetime'].dt.month_name()

# Define the desired order of months
month_order = ['January', 'February', 'March', 'April', 'May', 'June',
               'July', 'August', 'September', 'October', 'November', 'December']

# Convert the 'pickup_month' and 'dropoff_month' columns to categorical type with the specified order
cleaneddata['pickup_month'] = pd.Categorical(cleaneddata['pickup_month'], categories=month_order, ordered=True)
cleaneddata['dropoff_month'] = pd.Categorical(cleaneddata['dropoff_month'], categories=month_order, ordered=True)

# Sort the DataFrame by the 'pickup_month' column
df = cleaneddata.sort_values('pickup_month')

This one is to seperate the time and date into two different columns. Just for easier analysis latewr on

I have this test to see if the trip was in one day or a few days like. I wanted to see if it was plausible .As in maybe the trip duration 

In [8]:
# def round_to_nearest_hour(time):
#     rounded_time = time.replace(minute=0, second=0)
#     return rounded_time

# df['dropoff_time'] = df['dropoff_time'].apply(round_to_nearest_hour)
# df['pickup_time'] = df['pickup_time'].apply(round_to_nearest_hour)

In [9]:
# Identify holidays
cal = calendar()
holidays = cal.holidays(start='2022-01-01', end='2022-12-31')
# Convert 'pickup_date' to datetime-like object
df['pickup_date'] = pd.to_datetime(df['pickup_date'])

# Check if the pickup date is a holiday and assign day types
df['holiday'] = df['pickup_date'].isin(holidays)
df['week'] = df['pickup_date'].dt.dayofweek
df.loc[df['week'] >= 5, 'day_type'] = "weekend"
df.loc[df['week'] < 5, 'day_type'] = "workday"
df.loc[df['holiday'] == True, 'day_type'] = "holiday"

In [10]:
# print(df.head())

In [11]:
# # Drop the 'holiday' and 'week' columns
# df = df.drop(['holiday', 'week'], axis=1)

# Define a function to assign time slots
def time_slots(x):
    if x.hour in range(6, 12):
        return 'Morning'
    elif x.hour in range(12, 17):
        return 'Afternoon'
    elif x.hour in range(17, 22):
        return 'Evening'
    else:
        return 'Late Night'

# Apply the time_slots function to 'tpep_pickup_datetime'
df['dropoff_timeslots'] = df['dropoff_time'].apply(time_slots)
df['pickup_timeslots'] = df['pickup_time'].apply(time_slots)

# Print the updated DataFrame
# print(df.head())

Ok below is seasons just to make sure I have all the seasons here so that I can analysis the data.

In [12]:
# df.dtypes

In [13]:
# dataset_year = pd.to_datetime(df['tpep_pickup_datetime']).dt.year.unique()[0]

# winter_start = pd.to_datetime('2022-01-01').date()
# winter_end = pd.to_datetime('2022-02-28').date()
# spring_start = pd.to_datetime('2022-03-01').date()
# spring_end = pd.to_datetime('2022-05-31').date()
# summer_start = pd.to_datetime('2022-06-01').date()
# summer_end = pd.to_datetime('2022-08-31').date()
# autumn_start = pd.to_datetime('2022-09-01').date()
# autumn_end = pd.to_datetime('2022-11-30').date()

# # Set the 'season' value based on the date range
# df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.date
# df['season'] = pd.NA
# df.loc[(df['tpep_pickup_datetime'] >= winter_start) & (df['tpep_pickup_datetime'] <= winter_end), 'season'] = 'winter'
# df.loc[(df['tpep_pickup_datetime'] >= spring_start) & (df['tpep_pickup_datetime'] <= spring_end), 'season'] = 'spring'
# df.loc[(df['tpep_pickup_datetime'] >= summer_start) & (df['tpep_pickup_datetime'] <= summer_end), 'season'] = 'summer'
# df.loc[(df['tpep_pickup_datetime'] >= autumn_start) & (df['tpep_pickup_datetime'] <= autumn_end), 'season'] = 'autumn'

# # Filter the DataFrame for each season
# winter_data = df[df['season'] == 'winter']
# spring_data = df[df['season'] == 'spring']
# summer_data = df[df['season'] == 'summer']
# autumn_data = df[df['season'] == 'autumn']

# # Count the number of NaN values in the 'season' column
# nan_counts = df['season'].isna().sum()

# # Print the results
# print("Number of NaN values in 'season' column:", nan_counts)

## THE ONE BELOW SHOULD WORK

In [14]:
# Extract the month from the pickup datetime column
df['pickup_month'] = pd.to_datetime(df['tpep_pickup_datetime']).dt.month_name()

# Set the 'season' value based on the month
df['season'] = pd.NA
df.loc[df['pickup_month'].isin(['December', 'January', 'February']), 'season'] = 'winter'
df.loc[df['pickup_month'].isin(['March', 'April', 'May']), 'season'] = 'spring'
df.loc[df['pickup_month'].isin(['June', 'July', 'August']), 'season'] = 'summer'
df.loc[df['pickup_month'].isin(['September', 'October', 'November']), 'season'] = 'autumn'

# Filter the DataFrame for each season
winter_data = df[df['season'] == 'winter']
spring_data = df[df['season'] == 'spring']
summer_data = df[df['season'] == 'summer']
autumn_data = df[df['season'] == 'autumn']

# Count the number of NaN values in the 'season' column
nan_counts = df['season'].isna().sum()

# Print the results
print("Number of NaN values in 'season' column:", nan_counts)

Number of NaN values in 'season' column: 0


In [15]:
# # Get the unique values for the "passenger" column
# passenger_values = df['passenger_count'].unique()

# # Print the unique values
# print("Unique passenger values:")
# print(passenger_values)

# # Count the occurrences of each unique value in the "passenger" column
# passenger_counts = df['passenger_count'].value_counts()

# # Print the counts
# print("Value counts for passenger:")
# print(passenger_counts)

In [16]:
# passenger_counts = df['passenger_count'].value_counts()

# # Calculate the percentage for each value
# passenger_percentage = passenger_counts / len(df) * 100

# # Print the percentages
# print("Percentage for each passenger count:")
# print(passenger_percentage)

In [17]:
# print(df.dtypes) 

In [18]:
# Assuming 'combined_df' is your DataFrame and 'column1', 'column2', 'column3' are the columns you want to drop
combination = df.drop(['trip_distance','VendorID', 'tolls_amount', 'extra','tolls_amount','improvement_surcharge','total_amount','congestion_surcharge','airport_fee','store_and_fwd_flag','tip_amount','mta_tax','payment_type','fare_amount','RatecodeID'], axis=1)

In [ ]:
# print(combination)

In [ ]:
# nan_counts = combination.isna().sum()
# print("NaN counts for each column:")
# print(nan_counts)

In [ ]:
# Drop rows with any missing values
combination = combination.dropna()

In [ ]:
# print(combination.columns)
filtered_table = pa.Table.from_pandas(combination)
pq.write_table(filtered_table, 'dataforanalysing.parquet')